# EDA Datset Grammys

- In this Jupyter notebook, you will see the entire process of extraction, cleaning, and loading of one of the 2 datasets of the project, in this case the Grammy dataset which is extracted from a database to be processed in this notebook. At the end of all its transformation, it will be saved into a pickle file that will allow us to interact with it from other notebooks.
- The decision to use the pickle library was made for its ease of use and functionality in this Jupyter environment and Python programming language.

## Library Importation
### Used for Database Connection and Management
- Sqlalchemy and create_engine: These libraries help us to interact with our MySQL database from language files and assist us with the process of creating connections and queries to the tables of our databases.
- Dotenv, load_dotenv, and os: Libraries that assist us in handling credentials in public files like this notebook. Since we are using environment variables for the connection with MySQL, these libraries help us to use our credentials by calling them from a ".env" file.

### Used for Exploratory and Statistical Analysis of the Dataset
- Pandas and Matplotlib: High-performance libraries for data handling and exploratory data analysis, allowing for thorough and organized cleaning of our datasets. Additionally, they provide a wide range of statistical and graphical analysis capabilities, complemented by Matplotlib.
- Pickle: A widely used library for data transfer and processes within the Python language, offering flexibility and easy data persistence.

In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pickle
import pymysql

## Loading Environment Variables (Variables that will allow us to connect to our database)

In [3]:
load_dotenv()


True

In [4]:

db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_database = os.getenv("DB_DATABASE")

## Connection Established with PostgreSQL using SQLAlchemy
- As part of the code, we aim for it to return the header and information of the requested table at the end to verify that the process was successful

In [5]:
try:
    Workshop_2_mysql_connection_str = f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_database}'
    Workshop_2_mysql_db_connection = create_engine(Workshop_2_mysql_connection_str)
    print("Connection established with the database...")

    table_name = 'grammy_awards'

    query = f'SELECT * FROM {table_name}'

    grammy_awards_df = pd.read_sql(query, con=Workshop_2_mysql_db_connection)

    print(grammy_awards_df.head())
    print(grammy_awards_df.info())

except Exception as e:
    print(f"Failed to connect to the MySQL database: {e}")

finally:
    if 'Workshop_2_mysql_db_connection' in locals():
        Workshop_2_mysql_db_connection.dispose()
    print("Data connection and loading successful, connection closed.")



Connection established with the database...
   year                              title               published_at  \
0  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
1  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
2  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
3  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
4  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   

                  updated_at            category     nominee         artist  \
0  2020-05-19T05:10:28-07:00  Record Of The Year     Bad Guy  Billie Eilish   
1  2020-05-19T05:10:28-07:00  Record Of The Year     Hey, Ma       Bon Iver   
2  2020-05-19T05:10:28-07:00  Record Of The Year     7 rings  Ariana Grande   
3  2020-05-19T05:10:28-07:00  Record Of The Year  Hard Place         H.E.R.   
4  2020-05-19T05:10:28-07:00  Record Of The Year        Talk         Khalid   

                                          


## Search for Null Values
- Similarly to the Spotify dataset, we can see that some fields are detected as null. These fields are 'nominee', 'artist', 'workers', 'img'.

In [6]:
null_counts = grammy_awards_df.isnull().sum()
print("Null Value Counts per Column:")
print(null_counts)


Null Value Counts per Column:
year               0
title              0
published_at       0
updated_at         0
category           0
nominee            6
artist          1840
workers         2190
img             1367
winner             0
dtype: int64


## Removal of Records with Null Values in the Fields ('nominee', 'artist', 'workers', 'img')

In [7]:
grammy_awards_df.dropna(subset=['nominee', 'artist', 'workers', 'img'], inplace=True)

print(grammy_awards_df.isnull().sum())
print(grammy_awards_df.info())


year            0
title           0
published_at    0
updated_at      0
category        0
nominee         0
artist          0
workers         0
img             0
winner          0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 834 entries, 0 to 4629
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          834 non-null    int64 
 1   title         834 non-null    object
 2   published_at  834 non-null    object
 3   updated_at    834 non-null    object
 4   category      834 non-null    object
 5   nominee       834 non-null    object
 6   artist        834 non-null    object
 7   workers       834 non-null    object
 8   img           834 non-null    object
 9   winner        834 non-null    int64 
dtypes: int64(2), object(8)
memory usage: 71.7+ KB
None


## 'published_at' and 'updated_at' Fields
- These fields have records that correspond to dates but are being treated as 'object' type data.
- Let's analyze a bit more about how the records in these fields are structured.

In [8]:
print("Registros en el campo 'published_at':")
print(grammy_awards_df['published_at'])

print("\nRegistros en el campo 'updated_at':")
print(grammy_awards_df['updated_at'])


Registros en el campo 'published_at':
0       2020-05-19T05:10:28-07:00
1       2020-05-19T05:10:28-07:00
2       2020-05-19T05:10:28-07:00
3       2020-05-19T05:10:28-07:00
4       2020-05-19T05:10:28-07:00
                  ...            
4449    2017-11-28T00:03:45-08:00
4450    2017-11-28T00:03:45-08:00
4492    2017-11-28T00:03:45-08:00
4493    2017-11-28T00:03:45-08:00
4629    2017-11-28T00:03:45-08:00
Name: published_at, Length: 834, dtype: object

Registros en el campo 'updated_at':
0       2020-05-19T05:10:28-07:00
1       2020-05-19T05:10:28-07:00
2       2020-05-19T05:10:28-07:00
3       2020-05-19T05:10:28-07:00
4       2020-05-19T05:10:28-07:00
                  ...            
4449    2019-09-10T01:07:37-07:00
4450    2019-09-10T01:07:37-07:00
4492    2019-09-10T01:06:59-07:00
4493    2019-09-10T01:06:59-07:00
4629    2019-09-10T01:09:02-07:00
Name: updated_at, Length: 834, dtype: object



## Changing Type from 'object' to 'datetime' for the Fields 'published_at' and 'updated_at'

In [9]:
grammy_awards_df['published_at'] = pd.to_datetime(grammy_awards_df['published_at'], utc=True)
grammy_awards_df['updated_at'] = pd.to_datetime(grammy_awards_df['updated_at'], utc=True)
print(grammy_awards_df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 834 entries, 0 to 4629
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   year          834 non-null    int64              
 1   title         834 non-null    object             
 2   published_at  834 non-null    datetime64[ns, UTC]
 3   updated_at    834 non-null    datetime64[ns, UTC]
 4   category      834 non-null    object             
 5   nominee       834 non-null    object             
 6   artist        834 non-null    object             
 7   workers       834 non-null    object             
 8   img           834 non-null    object             
 9   winner        834 non-null    int64              
dtypes: datetime64[ns, UTC](2), int64(2), object(6)
memory usage: 71.7+ KB
None


## Analysis of the 'winner' Field
- We can see that the field only has one value, which is "1", which in the context of 'winner' probably indicates that the song or nominee was a winner. However, in this case, the 'winner' field may not be adding much value if all records categorize them as winners. Throughout the workshop and the exploratory data analysis (EDA) phase, we will analyze the true importance of the field and decide whether to keep it or remove it from the dataset.

In [16]:
winners_count = grammy_awards_df['winner'].value_counts()

print("Unique values of 'winner':")
print(winners_count)


Unique values of 'winner':
winner
1    834
Name: count, dtype: int64


In [17]:
unique_values = grammy_awards_df['winner'].unique()

print("Unique values in the 'winner' column:", unique_values)


Unique values in the 'winner' column: [1]


In [18]:
record_counts = grammy_awards_df['winner'].value_counts()

for value, count in record_counts.items():
    print(f"Total records with value '{value}': {count}")


Total records with value '1': 834


## Saving Cleaned Dataset as a Pickle File

In [19]:
with open('grammy_awards_df.pkl', 'wb') as f:
    pickle.dump(grammy_awards_df, f)



## Alternative Saving to CSV
- We include this line for anyone who wants to work with the cleaned dataframes as CSV files. However, remember that the merge process was facilitated by saving the Grammy and Spotify datasets as pickle files, allowing for seamless dataset handling across different notebooks.

In [20]:
#Save the DataFrame to a CSV file
#grammy_awards_df.to_csv("cleaned_grammy_awards.csv", index=False)